In [51]:
import pandas as pd

def head_cleaning (df) :
    h1 = df.iloc[0].astype(str)
    h2 = df.iloc[1].astype(str)
    header = df.columns.astype(str)

    new_columns = []

    for a, b, c in zip(header, h1, h2):
        # keep non-empty, non-"nan" parts
        parts=[a,b,c]
        parts = [p for p in parts if p.lower() != 'nan' and p != '']
        if len(parts) == 3:
            name = f"{parts[0]}_({parts[1]},{parts[2]})"
        elif len(parts) == 2:
            name = f"{parts[0]}_({parts[1]})"
        else :
            name = str(parts[0])
        new_columns.append(name)
    new_columns
    df.columns = new_columns
    df = df.rename(columns={new_columns[0]:"track_id"})[2:]
    return df



features=head_cleaning(pd.read_csv("data/fma_metadata/features.csv"))
echonest=head_cleaning(pd.read_csv("data/fma_metadata/echonest.csv"))
genres=pd.read_csv("data/fma_metadata/genres.csv")
tracks=pd.read_csv("data/fma_metadata/tracks.csv")

/tmp/ipykernel_3108/511543532.py:28: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,26

In [ ]:
"""
GPT summary of features : 

✅ 1. Chroma Features (chroma_stft, chroma_cqt, chroma_cens)

Chroma features describe the intensity of the 12 pitch classes (C, C#, D, …, B) regardless of octave. They are widely used in music analysis, genre classification, chord recognition, and tonal similarity.

✔️ chroma_stft

Computed using the short-time Fourier transform.
Represents how much energy each pitch class has in each short time window.

✔️ chroma_cqt

Computed using the constant-Q transform, which has a logarithmic frequency scale aligned with music notes.
More musically natural and stable for harmonic content.

✔️ chroma_cens

“Chroma Energy Normalized Statistics.”
A smoothed, more robust version of chroma designed to be invariant to dynamics, emphasizing harmonic patterns rather than exact amplitudes.
Useful for similarity, cover-song detection, and noisy conditions.

✔️ Why 12 values for each statistic?

Because each chroma vector has 12 bins (one per pitch class).
You then compute summary statistics:

max: peak energy of each pitch class

mean: overall average pitch-class energy

median: typical central value

min: lowest energy

std: variability

skew: asymmetry in distribution

kurtosis: how “peaked” or “heavy-tailed” the distribution is

✅ 2. MFCCs (Mel-Frequency Cepstral Coefficients)

MFCCs summarize the spectral envelope—a compact representation of timbre.
librosa typically produces 20 MFCC coefficients, each capturing a different aspect of the sound’s spectral shape.

You computed for each of the 20 coefficients:

kurtosis

max

mean

median

min

skew

std

✔️ What MFCCs represent:

Lower MFCCs → overall spectral slope (brightness/darkness)

Higher MFCCs → fine-grained spectral details (timbre texture)

Widely used in speech recognition, emotion detection, speaker identification, and music genre classification.

✅ 3. RMSE (Root Mean Square Energy)

Measures signal energy over time.

Summary statistics represent:

highest energy (max)

average loudness (mean)

typical loudness (median)

dynamic range (min, std)

shape of the energy distribution (skew, kurtosis)

Essential for tasks involving onset detection, loudness, and dynamics.

✅ 4. Spectral Features

These describe the frequency content of the audio:

✔️ spectral_centroid

“Center of mass” of the spectrum → brightness of the sound.

✔️ spectral_bandwidth

Spread of the spectrum around the centroid → spectral width.

✔️ spectral_rolloff

Frequency below which a percentage (usually 85%) of spectral energy lies → relates to high-frequency content.

Each has stats: max, mean, median, min, skew, kurtosis, std.

✅ 5. Spectral Contrast

Spectral contrast measures the difference between peaks and valleys in the spectrum across 7 frequency bands.

Interpretation:

High contrast → strong harmonics or clear musical notes

Low contrast → noisy, broadband signals

You computed 7-band statistics for each of:

kurtosis

max

mean

median

min

skew

std

This yields many columns because spectral contrast produces 7 numbers per frame.

✅ 6. Tonnetz (Tonal Centroid Features)

Derived from harmonic analysis; describes relationships among pitch classes based on music theory (circle of fifths, major/minor modes).

Tonnetz gives 6 dimensions, which relate to:

consonance/dissonance

harmonic progression

tonal stability

You computed:

kurtosis, max, mean, median, min, skew, std for each of the 6 tonnetz dimensions.

Used in genre classification, chord recognition, mood/emotion prediction.

✅ 7. ZCR (Zero Crossing Rate)

Counts how often the signal crosses zero → relates to noisiness or high-frequency content.

High ZCR → fricatives in speech, noisy instruments

Low ZCR → voiced speech, smoother tones

You extracted summary stats: max, mean, median, min, skew, kurtosis, std.

🎯 In summary

These features capture nearly every dimension of the audio signal:

Category	What it describes
Chroma family	Harmonic & pitch-class energy
MFCCs	Timbre (spectral envelope)
RMSE	Loudness & dynamics
Spectral centroid/bandwidth/rolloff	Frequency distribution & brightness
Spectral contrast	Harmonic peaks vs valleys
Tonnetz	Tonal relationships (music theory)
ZCR	Noisiness & high-frequency activity
"""

track_id
chroma_cens_(kurtosis,01)
chroma_cens.1_(kurtosis,02)
chroma_cens.2_(kurtosis,03)
chroma_cens.3_(kurtosis,04)
chroma_cens.4_(kurtosis,05)
chroma_cens.5_(kurtosis,06)
chroma_cens.6_(kurtosis,07)
chroma_cens.7_(kurtosis,08)
chroma_cens.8_(kurtosis,09)
chroma_cens.9_(kurtosis,10)
chroma_cens.10_(kurtosis,11)
chroma_cens.11_(kurtosis,12)
chroma_cens.12_(max,01)
chroma_cens.13_(max,02)
chroma_cens.14_(max,03)
chroma_cens.15_(max,04)
chroma_cens.16_(max,05)
chroma_cens.17_(max,06)
chroma_cens.18_(max,07)
chroma_cens.19_(max,08)
chroma_cens.20_(max,09)
chroma_cens.21_(max,10)
chroma_cens.22_(max,11)
chroma_cens.23_(max,12)
chroma_cens.24_(mean,01)
chroma_cens.25_(mean,02)
chroma_cens.26_(mean,03)
chroma_cens.27_(mean,04)
chroma_cens.28_(mean,05)
chroma_cens.29_(mean,06)
chroma_cens.30_(mean,07)
chroma_cens.31_(mean,08)
chroma_cens.32_(mean,09)
chroma_cens.33_(mean,10)
chroma_cens.34_(mean,11)
chroma_cens.35_(mean,12)
chroma_cens.36_(median,01)
chroma_cens.37_(median,02)
chroma_cens.3